## This code is automating webscraping using Beautiful Soup and Selenium where it is automating and filling a form by itself and then opening the webpage and scraping the data. At the end it stores the data in a dataframe which can be seen in the code below. 

The whole process of data automation is visible during the process. 

In [1]:
import camelot
import pandas as pd

In [4]:
import os
from ctypes.util import find_library
import camelot

# Set Ghostscript path
os.environ['PATH'] += os.pathsep + '/opt/homebrew/bin'
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib'

# Verify that Python can find Ghostscript
print(find_library("gs"))


/opt/homebrew/lib/libgs.dylib


In [37]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [29]:
PATH = "/Users/windows/Desktop/Drivers/chromedriver"
options = webdriver.ChromeOptions()

service = Service(ChromeDriverManager().install())

In [30]:
driver = webdriver.Chrome(service = service, options = options)

In [46]:
url = 'https://egramswaraj.gov.in/actionPlanReportForGP.do'

In [47]:
driver.get(url)

In [48]:
print(driver.title)

Approve Plan For GP New


In [49]:
# Define a wait
wait = WebDriverWait(driver, 20)


In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

# Initializing the WebDriver
driver = webdriver.Chrome()

# Opening the main website
driver.get('https://egramswaraj.gov.in/actionPlanReportForGP.do')

# time wait to fill the form
wait = WebDriverWait(driver, 10)

# Step 1 - Selecting Plan Year
plan_year_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'fyear')))
plan_year_dropdown.click()
plan_year_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='fyear']/option[@value='2024']")))
plan_year_option.click()

# Step 2- Selecting State
state_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'stateCode')))
state_dropdown.click()
state_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='stateCode']/option[@value='28']")))  # Andhra Pradesh
state_option.click()

# Step 3- Selecting Panchayat Level with a time lag of 2 seconds so that the form loads
time.sleep(2)
panchayat_level_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'panchayatLevel')))
panchayat_level_dropdown.click()
panchayat_level_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='panchayatLevel']/option[@value='3']")))  # Grama Panchayat
panchayat_level_option.click()


# Step 4- Selecting District Panchayat & Equivalent with a time lag of 2 seconds so that the form loads
time.sleep(2)
district_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'districtId')))
district_dropdown.click()
district_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='districtId']/option[@value='475']")))  # Visakhapatanam
district_option.click()

# Step 5- Selecting Block Panchayat & Equivalent with a time lag of 2 seconds so that the form loads
time.sleep(2)
block_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'blockId')))
block_dropdown.click()
block_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='blockId']/option[@value='5917']")))  # Anandapuram
block_option.click()

# Step 6- Selecting Gram Panchayat & Equivalent with a time lag of 2 seconds so that the form loads
time.sleep(2)
gp_dropdown = wait.until(EC.element_to_be_clickable((By.ID, 'gpId')))
gp_dropdown.click()
gp_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='gpId']/option[@value='211707']")))  # Anandapuram
gp_option.click()

# Step 7- 10 seconds to manually fill the captcha code
time.sleep(20)

# Step 8- Automatic clicking "Get Report" button
get_report_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@onclick='submitProfileReport()']")))
get_report_button.click()

# 10 seconds wait to observe the selected values
time.sleep(10)

# Close the driver
#driver.quit()

#Till here, we have opened the website that we want to scrape. 
# After this we will be scrolling and scraping the required data. 

# Scroll down to ensure all elements are loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2) 

# Get the page source and parse it with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Extract the first table
table1 = soup.find_all('table', {'class': 'table table-striped table-bordered table-hover w-100'})[0]
headers1 = [th.text.strip() for th in table1.find_all('th')]
rows1 = table1.find_all('tr')[1:]  # Skip the header row

data1 = []
for row in rows1:
    cells = row.find_all('th') + row.find_all('td')  # Include both <th> and <td> elements
    cells_text = [cell.text.strip() for cell in cells]
    if len(cells_text) == 5:  # Ensure the correct number of columns
        data1.append(cells_text)

df1 = pd.DataFrame(data1, columns=headers1[:5])  # Use only the first 5 headers

# Extract the second table
table2 = soup.find('table', {'id': 'myTable'})
headers2 = [th.text.strip() for th in table2.find_all('th')]
rows2 = table2.find_all('tr')[1:]  # Skip the header row

data2 = []
for row in rows2:
    cells = row.find_all('td')
    cells_text = [cell.text.strip() for cell in cells]
    if cells_text:
        data2.append(cells_text)

df2 = pd.DataFrame(data2, columns=headers2)
df2

# Excel file with separate sheets
with pd.ExcelWriter('/Users/akshitakhajuria/Downloads/extracted2.xlsx') as writer:
    df1.to_excel(writer, sheet_name='Table 1', index=False)
    df2.to_excel(writer, sheet_name='Table 2', index=False)

# Close the driver
driver.quit()

print("Tables have been extracted and saved to 'extracted_tables.xlsx'")

Tables have been extracted and saved to 'extracted_tables.xlsx'


In [73]:
df1

,Plan Year,State Name,District Panchayat & Equivalent,Block Panchayat and Equivalent,Gram Panchayat & Equivalent
0,2024-2025,Andhra Pradesh,Visakhapatanam,Anandapuram,Anandapuram


In [74]:
df2

,S.No.,Activity Code,Activity Name,Activity Cost,Focus Area,Activity Type,Activity Status,Activity Output Type,Plan Type
0,1,105477918,Repair of pipe drinking water,190000,Drinking water,Public Works,Activity Approved,Maintenance,Part of Main Plan
1,2,105477642,Operation and Maintenance of drinking/piped wa...,60000,Drinking water,Public Works,Activity Approved,Operational,Part of Main Plan
2,3,105394130,Creation of a new source of drinking water,255477,Drinking water,Public Works,Activity Approved,Asset,Part of Main Plan
3,4,105393327,Construction of new Piped Water Supply schemes,200000,Drinking water,Public Works,Activity Approved,Asset,Part of Main Plan
4,5,105341533,Construction of new Piped Water Supply schemes,115657,Drinking water,Public Works,Activity Approved,Asset,Part of Main Plan
...,...,...,...,...,...,...,...,...,...
120,121,92729414,Any other operation and maintenance needs rela...,100000,Sanitation,Public Works,Activity Pending for Inclusion,Operational,
121,122,92726020,Technical & administrative expenses,100,Drinking water,Public Works,Activity Pending for Inclusion,Operational,
122,123,92726917,Payment of honorarium to swachhagrahis,504000,Sanitation,Public Works,Activity Pending for Inclusion,Operational,
123,124,92725373,Operation and Maintenance of drinking/piped wa...,200000,Drinking water,Public Works,Activity Pending for Inclusion,Operational,
